### Multi-source data-derived elevation changes fusion by using error-inverse weighting method.

In [1]:
from utils.geotif_io import readTiff
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.colors import LinearSegmentedColormap
import pandas as pd
import xarray as xr
import numpy as np
import os
from utils.ransac_fitting import ransac_fitting


In [2]:
path_ele_change_isat1 = 'data/ele-change-result/ele_change_isat1_tiles_sub.nc'
path_ele_change_isat2 = 'data/ele-change-result/ele_change_isat2_tiles_sub.nc'
path_ele_change_cryo2 = 'data/ele-change-result/ele_change_cryo2_tiles_sub.nc'
path_ele_change_dems = 'data/ele-change-result/ele_change_dems_tiles_sub.nc'
### Write out
path_result_fused = 'data/ele-change-result/ele_change_fused_tiles_sub.nc'



In [3]:
ele_change_isat1_xr = xr.open_dataset(path_ele_change_isat1)
ele_change_isat2_xr = xr.open_dataset(path_ele_change_isat2)
ele_change_cryo2_xr = xr.open_dataset(path_ele_change_cryo2)
ele_change_dems_xr = xr.open_dataset(path_ele_change_dems)
tiles_id_dems, years_dems = ele_change_dems_xr.tiles_id.values, ele_change_dems_xr.years.values
ele_change_isat1_xr
ele_change_cryo2_xr
ele_change_dems_xr



<xarray.Dataset>
Dimensions:                      (tiles_id: 112, bins_id: 50, years: 23)
Coordinates:
  * tiles_id                     (tiles_id) object 'tile_270_960' ... 'tile_3...
  * bins_id                      (bins_id) object '2500-2600' ... '7400-7500'
  * years                        (years) object '2000' '2001' ... '2021' '2022'
Data variables: (12/15)
    tiles_lat                    (tiles_id) int64 ...
    tiles_lon                    (tiles_id) int64 ...
    area_glacier_bins            (bins_id) float64 ...
    area_glacier_tiles           (tiles_id) float64 ...
    ele_change_cor_setp          (years) float64 ...
    ele_change_cor_error_setp    (years) float64 ...
    ...                           ...
    ele_change_cor_tiles         (tiles_id, years) float64 ...
    rate_tiles                   (tiles_id) float64 ...
    rate_error_tiles             (tiles_id) float64 ...
    ele_change_cor_bins          (bins_id, years) float64 ...
    rate_bins                    (bins_id) float64 ...
    rate_error_bins              (bins_id) float64 ...

In [4]:
print('Elevation change rate by aster dems: %f +/- %f' % (ele_change_dems_xr['rate_setp'], ele_change_dems_xr['rate_error_setp']))
print('Elevation change rate by icesat-1: %f +/- %f' % (ele_change_isat1_xr['rate_setp'], ele_change_isat1_xr['rate_error_setp']))
print('Elevation change rate by icesat-2: %f +/- %f' % (ele_change_isat2_xr['rate_setp'], ele_change_isat2_xr['rate_error_setp']))
print('Elevation change rate by cryosat-2: %f +/- %f' % (ele_change_cryo2_xr['rate_setp'], ele_change_cryo2_xr['rate_error_setp']))



Elevation change rate by aster dems: -0.603805 +/- 0.296360
Elevation change rate by icesat-1: -1.179263 +/- 0.440122
Elevation change rate by icesat-2: -0.336730 +/- 0.117726
Elevation change rate by cryosat-2: -1.065336 +/- 0.108559


### Fused elevation change for overall setp region.


In [5]:
ele_change_setp_years = []
for year in years_dems:
    ele_changes = []
    ele_changes_error = []
    ele_change_dem = ele_change_dems_xr.sel(years=year).ele_change_cor_setp.values
    ele_changes.append(ele_change_dem)
    if year in ele_change_isat1_xr.years.values:
        ele_change_isat1 = ele_change_isat1_xr.sel(years=year).ele_change_cor_setp.values
        ele_changes.append(ele_change_isat1)
    if year in ele_change_isat2_xr.years.values:
        ele_change_isat2 = ele_change_isat2_xr.sel(years=year).ele_change_cor_setp.values
        ele_changes.append(ele_change_isat2)
    if year in ele_change_cryo2_xr.years.values:
        ele_change_cryo2 = ele_change_cryo2_xr.sel(years=year).ele_change_cor_setp.values
        ele_changes.append(ele_change_cryo2)
    ele_change_setp_years.append(np.mean(ele_changes))
ele_change_setp_years



[6.31841744149424,
 3.1876326223142044,
 3.2487930425106617,
 2.945242040636537,
 3.3392576508563474,
 2.437758767481651,
 2.281226586225385,
 -2.640467632124819,
 0.5144922517650798,
 0.27972287923939154,
 -1.3790057509453626,
 -2.529328406009042,
 -2.4341205609703005,
 -4.608163224948203,
 -5.253778524068105,
 -4.742414905145945,
 -4.170804218057426,
 -6.141245820536003,
 -7.630598950370767,
 -7.295067968292044,
 -9.547313137068363,
 -10.56315834485595,
 -10.623689187534106]

In [6]:
ele_change_setp_years

[6.31841744149424,
 3.1876326223142044,
 3.2487930425106617,
 2.945242040636537,
 3.3392576508563474,
 2.437758767481651,
 2.281226586225385,
 -2.640467632124819,
 0.5144922517650798,
 0.27972287923939154,
 -1.3790057509453626,
 -2.529328406009042,
 -2.4341205609703005,
 -4.608163224948203,
 -5.253778524068105,
 -4.742414905145945,
 -4.170804218057426,
 -6.141245820536003,
 -7.630598950370767,
 -7.295067968292044,
 -9.547313137068363,
 -10.56315834485595,
 -10.623689187534106]

In [7]:

ele_change_filtered, ele_change_fitting, rate_setp = ransac_fitting(x=np.arange(len(ele_change_setp_years)), y=np.array(ele_change_setp_years), thre_mask=50)
ele_change_2000_2012_filtered, ele_change_2000_2012_fitting, rate_2000_2012_setp = ransac_fitting(x=np.arange(len(ele_change_setp_years[0:-10])), y=np.array(ele_change_setp_years[0:-10]), thre_mask=50)
ele_change_2013_2022_filtered, ele_change_2013_2022_fitting, rate_2013_2022_setp = ransac_fitting(x=np.arange(len(ele_change_setp_years[-10:])), y=np.array(ele_change_setp_years[-10:]), thre_mask=50)

#### Uncertainty for 2000-2022
epsilon = ele_change_setp_years - ele_change_fitting
sigma_dh = np.std(epsilon)
sigma_dh_dt_setp = sigma_dh/len(ele_change_setp_years)
print('sigma_dh:', sigma_dh)
print('Elevation change rate:', rate_setp)
print('sigma_dh_dt:', sigma_dh_dt_setp)
print('....2000-2012')
#### Uncertainty for 2013-2022
epsilon_2000_2012 = ele_change_setp_years[0:-10] - ele_change_2000_2012_fitting
sigma_dh_2000_2012 = np.std(epsilon_2000_2012)
sigma_dh_dt_2000_2012_setp = sigma_dh_2000_2012/len(ele_change_setp_years)
print('sigma_dh_2000_2012:', sigma_dh_2000_2012)
print('Elevation change rate(2000_2012):', rate_2000_2012_setp)
print('sigma_dh_dt_2000_2012:', sigma_dh_dt_2000_2012_setp)
print('....2013-2022')
#### Uncertainty for 2013-2022
epsilon_2013_2022 = ele_change_setp_years[-10:] - ele_change_2013_2022_fitting
sigma_dh_2013_2022 = np.std(epsilon_2013_2022)
sigma_dh_dt_2013_2022_setp = sigma_dh_2013_2022/len(ele_change_setp_years)
print('sigma_dh_2013_2022:', sigma_dh_2013_2022)
print('Elevation change rate(2013_2022):', rate_2013_2022_setp)
print('sigma_dh_dt_2013_2022:', sigma_dh_dt_2013_2022_setp)


sigma_dh: 1.0555330788745347
Elevation change rate: -0.7095110518525167
sigma_dh_dt: 0.04589274255976238
....2000-2012
sigma_dh_2000_2012: 1.1892394518897835
Elevation change rate(2000_2012): -0.5828590023298202
sigma_dh_dt_2000_2012: 0.051706063125642764
....2013-2022
sigma_dh_2013_2022: 0.8438227825844199
Elevation change rate(2013_2022): -0.7648001693875077
sigma_dh_dt_2013_2022: 0.036687947068887825


### Fused elevation change for tiles.

In [8]:
import warnings

ele_change_tiles_years = np.zeros_like(ele_change_dems_xr.ele_change_cor_tiles)
ele_change_tiles_years[:] = np.nan
for i_year, year in enumerate(years_dems):
    for i_tile, tile in enumerate(tiles_id_dems):
        ele_changes_tile_year = []
        ele_change_dem_tile_year = ele_change_dems_xr.sel(years=year).sel(tiles_id=tile).ele_change_cor_tiles.values
        ele_changes_tile_year.append(ele_change_dem_tile_year)
        if year in ele_change_isat1_xr.years.values and tile in ele_change_isat1_xr.tiles_id.values:
            ele_change_tile_isat1 = ele_change_isat1_xr.sel(years=year).sel(tiles_id=tile).ele_change_cor_tiles.values
            ele_changes_tile_year.append(ele_change_tile_isat1)
        if year in ele_change_isat2_xr.years.values and tile in ele_change_isat2_xr.tiles_id.values:
            ele_change_tile_isat2 = ele_change_isat2_xr.sel(years=year).sel(tiles_id=tile).ele_change_cor_tiles.values
            ele_changes_tile_year.append(ele_change_tile_isat2)
        if year in ele_change_cryo2_xr.years.values and tile in ele_change_cryo2_xr.tiles_id.values:
            ele_change_tile_cryo2 = ele_change_cryo2_xr.sel(years=year).sel(tiles_id=tile).ele_change_cor_tiles.values
            ele_changes_tile_year.append(ele_change_tile_cryo2)
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            ele_changes_tile_year = np.nanmean(ele_changes_tile_year)
        ele_change_tiles_years[i_tile, i_year] = ele_changes_tile_year

ele_change_tiles_years.shape


(112, 23)

In [9]:
rate_tiles, rate_error_tiles = [], []
rate_2000_2012_tiles, rate_error_2000_2012_tiles = [], []
rate_2013_2022_tiles, rate_error_2013_2022_tiles = [], []

for i_tile, tile in enumerate(tiles_id_dems):
  ele_change_tile = ele_change_tiles_years[i_tile]
  ### 2000-2022
  ids_valid = np.where(~np.isnan(ele_change_tile))[0]
  if ids_valid.shape[0]>2:
    ele_change_filtered_tile, ele_change_fitting_tile, rate_tile = \
                                  ransac_fitting(x=np.arange(len(ele_change_tile)), y=ele_change_tile, thre_mask=50)
    rate_tiles.append(rate_tile)
    ## Uncertainty.
    epsilon = ele_change_tile - ele_change_fitting_tile
    sigma_dh = np.nanstd(epsilon)
    sigma_dh_dt = sigma_dh/len(ele_change_tile)
    rate_error_tiles.append(sigma_dh_dt)
  else: 
    rate_tiles.append(np.nan)
    rate_error_tiles.append(np.nan)

  ### 2000-2012
  ele_change_2000_2012_tile = ele_change_tiles_years[i_tile][0:-10]
  ids_valid_2000_2012 = np.where(~np.isnan(ele_change_2000_2012_tile))[0]
  if ids_valid_2000_2012.shape[0]>2:
    ele_change_filtered_2000_2012_tile, ele_change_fitting_2000_2012_tile, rate_2000_2012_tile = \
                              ransac_fitting(x=np.arange(len(ele_change_2000_2012_tile)), y=ele_change_2000_2012_tile, thre_mask=50)
    rate_2000_2012_tiles.append(rate_2000_2012_tile)
    ## Uncertainty.
    epsilon_2000_2012 = ele_change_2000_2012_tile - ele_change_fitting_2000_2012_tile
    sigma_dh_2000_2012 = np.nanstd(epsilon_2000_2012)
    sigma_dh_dt_2000_2012 = sigma_dh_2000_2012/len(ele_change_2000_2012_tile)
    rate_error_2000_2012_tiles.append(sigma_dh_dt_2000_2012)
  else: 
    rate_2000_2012_tiles.append(np.nan)
    rate_error_2000_2012_tiles.append(np.nan)

  ### 2013-2022
  ele_change_2013_2022_tile = ele_change_tiles_years[i_tile][-10:]
  ids_valid_2013_2022 = np.where(~np.isnan(ele_change_2013_2022_tile))[0]
  if ids_valid_2013_2022.shape[0]>2:
    ele_change_filtered_2013_2022_tile, ele_change_fitting_2013_2022_tile, rate_2013_2022_tile = \
                                  ransac_fitting(x=np.arange(len(ele_change_2013_2022_tile)), y=ele_change_2013_2022_tile, thre_mask=50)        
    rate_2013_2022_tiles.append(rate_2013_2022_tile)
    ## Uncertainty.
    epsilon_2013_2022 = ele_change_2013_2022_tile - ele_change_fitting_2013_2022_tile
    sigma_dh_2013_2022 = np.nanstd(epsilon_2013_2022)
    sigma_dh_dt_2013_2022 = sigma_dh_2013_2022/len(ele_change_2013_2022_tile)
    rate_error_2013_2022_tiles.append(sigma_dh_dt_2013_2022)
  else: 
    rate_2013_2022_tiles.append(np.nan)
    rate_error_2013_2022_tiles.append(np.nan)

rate_tiles
len(rate_error_tiles)




112

In [10]:
### 3) write out statistic of stable region to the xarray .nc file.
tiles_lat, tiles_lon = [], []
for tile_id in ele_change_dems_xr.tiles_id.values:
    tiles_lat.append(int(tile_id.split('_')[1]))
    tiles_lon.append(int(tile_id.split('_')[2]))

### Conver to xarray data.
result_fused_xr =xr.Dataset(
        {
        'tiles_lat': (["tiles_id"], tiles_lat),
        'tiles_lon': (["tiles_id"], tiles_lon),
        "area_glacier_tiles": (["tiles_id"], ele_change_dems_xr['area_glacier_tiles'].values),
        "ele_change_setp": (["years"], ele_change_setp_years),
        "ele_change_tiles": (['tiles_id', "years"], ele_change_tiles_years),
        'rate_setp': rate_setp,
        'rate_error_setp': sigma_dh_dt_setp, 
        'rate_2000_2012_setp': rate_2000_2012_setp,
        'rate_error_2000_2012_setp': sigma_dh_dt_2000_2012_setp, 
        'rate_2013_2022_setp': rate_2013_2022_setp,
        'rate_error_2013_2022_setp': sigma_dh_dt_2013_2022_setp, 
        'rate_tiles': (["tiles_id"], rate_tiles),
        'rate_error_tiles': (["tiles_id"], rate_error_tiles),
        'rate_2000_2012_tiles': (["tiles_id"], rate_2000_2012_tiles),
        'rate_error_2000_2012_tiles': (["tiles_id"], rate_error_2000_2012_tiles),
        'rate_2013_2022_tiles': (["tiles_id"], rate_2013_2022_tiles),
        'rate_error_2013_2022_tiles': (["tiles_id"], rate_error_2013_2022_tiles),
        },
        coords={'tiles_id': tiles_id_dems,
                'years': years_dems})

if os.path.exists(path_result_fused): os.remove(path_result_fused)
result_fused_xr.to_netcdf(path_result_fused)


